In [1]:
from tqdm import tqdm
from time import perf_counter
from pymilvus import MilvusClient
from data_load import load, embed

In [2]:
df, col, path = load()
df.head()

Data Shape: (72110, 6)


id,q_item_id,q_level1_global_be_category,q_item_name,q_link_first_image,q_item_name_clean
u32,i64,str,str,str,str
0,20837343532,"""Men Shoes""","""Giày Một vải Nước ngọt không d…","""https://cf.shopee.sg/file/vn-1…","""giày một vải nước ngọt không d…"
1,29902674276,"""Men Shoes""","""（100% Auth ）Giày Sneaker Nam N…","""https://cf.shopee.sg/file/sg-1…","""（100% auth ）giày sneaker nam n…"
2,28853113193,"""Men Shoes""","""/Hot/Giày thể thao *_Nike AirF…","""https://cf.shopee.sg/file/sg-1…","""/hot/giày thể thao *_nike airf…"
3,23436040086,"""Men Shoes""","""Khăn Lau Giày Đa Năng THE WIPE…","""https://cf.shopee.sg/file/vn-1…","""khăn lau giày đa năng the wipe…"
4,22561418198,"""Men Shoes""","""Cây giữ form giày chống nhăn d…","""https://cf.shopee.sg/file/vn-1…","""cây giữ form giày chống nhăn d…"


In [3]:
item = df['q_item_name_clean'].to_list()
embeddings, file_embed, df = embed(df, path)

In [4]:
client = MilvusClient("milvus_demo.db")

collection = 'demo_collection'
if client.has_collection(collection_name=collection):
    client.drop_collection(collection_name=collection)
    
client.create_collection(
    collection_name=collection,
    dimension=embeddings.shape[1],
    metric_type="IP"
)

In [5]:
data = df.to_dicts()

In [6]:
start = perf_counter()

total_sample = len(data)
batch_size = 500
num_batches = (total_sample + batch_size) // batch_size
print(f'Total batches: {num_batches}, Batch size: {batch_size:,.0f}')
for i, idx in tqdm(enumerate(range(num_batches), start=1), total=num_batches):
    start_idx = idx * batch_size
    end_idx = min(start_idx + batch_size, total_sample)
    batch = data[start_idx:end_idx]
    res = client.insert(collection_name="demo_collection", data=batch)
    
end = perf_counter() - start
print(f'Total Index Time: {end:,.0f}s')

Total batches: 145, Batch size: 500


100%|████████████████████████████████████████████████████| 145/145 [00:15<00:00,  9.24it/s]

Total Index Time: 16s


In [7]:
start = perf_counter()

batch_size = 100
num_batches = (total_sample + batch_size) // batch_size
print(f'Total batches: {num_batches}, Batch size: {batch_size:,.0f}')
for i, idx in tqdm(enumerate(range(num_batches), start=1), total=num_batches):
    start_idx = idx * batch_size
    end_idx = min(start_idx + batch_size, total_sample)
    batch = embeddings[start_idx:end_idx].tolist()
    res = client.search(
        collection_name=collection,
        data=batch,
        limit=10,
        search_params={"metric_type": "IP", "params": {}},
        output_fields=col
    )

end = perf_counter() - start
print(f'Total Query Time: {end:,.0f}s')

Total batches: 722, Batch size: 100


100%|████████████████████████████████████████████████████| 722/722 [01:16<00:00,  9.47it/s]

Total Query Time: 76s


In [10]:
df[start_idx:end_idx].head()

id,q_item_id,q_level1_global_be_category,q_item_name,q_link_first_image,q_item_name_clean,vector
u32,i64,str,str,str,str,"array[f32, 1024]"
72100,29352685061,"""Men Shoes""","""（100% Auth ）Giày Sneaker Nam N…","""https://cf.shopee.sg/file/sg-1…","""（100% auth ）giày sneaker nam n…","[0.039612, -0.024429, … 0.011963]"
72101,25680344702,"""Men Shoes""","""Bình xịt khử mùi giày,tất bình…","""https://cf.shopee.sg/file/vn-1…","""bình xịt khử mùi giày,tất bình…","[0.017593, -0.000049, … 0.007217]"
72102,5465959941,"""Men Shoes""","""Giày thể thao sneaker ultra bo…","""https://cf.shopee.sg/file/f200…","""giày thể thao sneaker ultra bo…","[0.015053, -0.012138, … 0.065979]"
72103,19089276484,"""Men Shoes""","""Xịt khử mùi hôi giày dép công …","""https://cf.shopee.sg/file/vn-1…","""xịt khử mùi hôi giày dép công …","[-0.017456, -0.049622, … 0.01416]"
72104,28402589440,"""Men Shoes""","""（100% Auth ）Giày Sneaker Nam N…","""https://cf.shopee.sg/file/sg-1…","""（100% auth ）giày sneaker nam n…","[-0.010597, -0.008514, … 0.000468]"


In [8]:
res[1]

[{'id': 72101,
  'distance': 0.9998304843902588,
  'entity': {'q_item_id': 25680344702,
   'q_level1_global_be_category': 'Men Shoes',
   'q_item_name': 'Bình xịt khử mùi giày,tất bình xịt thể thao khử mùi hôi chân thơm mát,kháng khuẩn',
   'q_link_first_image': 'https://cf.shopee.sg/file/vn-11134207-7r98o-lv92vrtokxrte1',
   'q_item_name_clean': 'bình xịt khử mùi giày,tất bình xịt thể thao khử mùi hôi chân thơm mát,kháng khuẩn'}},
 {'id': 17097,
  'distance': 0.9708941578865051,
  'entity': {'q_item_id': 26954717704,
   'q_level1_global_be_category': 'Men Shoes',
   'q_item_name': 'Giày nam Da bò Lacos da trơn màu đen bản da cực mềm và êm chân, có kèm ảnh tại shop ảnh khách thử',
   'q_link_first_image': 'https://cf.shopee.sg/file/vn-11134207-7r98o-lxf4462eu9qx7a',
   'q_item_name_clean': 'giày nam da bò lacos da trơn màu đen bản da cực mềm và êm chân, có kèm ảnh tại shop ảnh khách thử'}},
 {'id': 48462,
  'distance': 0.953707218170166,
  'entity': {'q_item_id': 11064010076,
   'q_lev